This is just the example

In [1]:
import random
from numpy.random import uniform
from numpy import multiply
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

from mesa import Model, Agent
from mesa.time import RandomActivation
from mesa.space import Grid
from mesa.datacollection import DataCollector
from mesa.batchrunner import BatchRunner

from numpy import multiply
from scipy.ndimage.filters import gaussian_filter

import pandas as pd
import matplotlib.cm as cm
from math import pi
from windrose import WindroseAxes

ImportError: No module named 'windrose'